# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [52]:
# imports
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import base64
from io import BytesIO
import requests
import gradio as gr

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
ollama_url = "http://localhost:11434/v1"

# set up environment
openai = OpenAI()
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [29]:
def system_prompt(selected_model):
    return f"""
    You are a tech expert and know every coding language, and can give 
    nice, detailed and simple explanations for the given questions.
    Introduce yourself by saying which model you are every time you answer. For example, this is {selected_model}.  
    """

def talker(message):
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    display(Audio(output_filename, autoplay=True))

def listener(audio_file):
    with open(audio_file, "rb") as audio:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio
        )
    return transcript.text

In [55]:
def chat(history, selected_model):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt(selected_model)}] + history

    if selected_model == "GPT-4o-mini":
        stream = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, stream=True)
        response = ""
        
        for chunk in stream:
            try:
                response += chunk.choices[0].delta.content or ''
                updated_history = history + [{"role": "assistant", "content": response}]
                # talker(response)
                yield updated_history  
            except Exception as e:
                print(f"Streaming error: {e}")
                yield "Sorry, there was an error processing your request."
    elif selected_model == "Llama3.2":
        stream = ollama.chat.completions.create(model="llama3.2", messages=messages, stream=True)        
        response = ""
        
        for chunk in stream:
            try:
                response += chunk.choices[0].delta.content or ''
                updated_history = history + [{"role": "assistant", "content": response}]
                # talker(response)
                yield updated_history  
            except Exception as e:
                print(f"Streaming error: {e}")
                yield "Sorry, there was an error processing your request."


In [ ]:
with gr.Blocks() as ui:

    gr.Markdown("## AI Code Assistant")
    gr.Markdown("**Select your preferred AI model:**")
    
    model_dropdown = gr.Dropdown(
        choices=["GPT-4o-mini", "Llama3.2"], 
        value="GPT-4o-mini",  # default selection
        label="Choose Model"
    )

    with gr.Row():
        chatbot = gr.Chatbot(height=200, type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        # Audio input for voice messages
        audio_input = gr.Audio(
            sources=["microphone", "upload"], 
            type="filepath", 
            label="🎙️ Voice Message"
        )
    with gr.Row():
        voice_submit = gr.Button("Send Voice Message", variant="secondary")
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    def process_voice_input(audio_file):
        """Convert voice to text and put it in the text box"""
        if audio_file is not None:
            transcribed_text = listener(audio_file)
            if transcribed_text and not transcribed_text.startswith("Error"):
                return transcribed_text
        return ""
    
    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[chatbot, model_dropdown], outputs=[chatbot]
    )

    voice_submit.click(
        process_voice_input,
        inputs=[audio_input],
        outputs=[entry]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)